In [ ]:
import folium
import json
import requests
import config

### Source

In [ ]:
points = {
    "a": "Европейский проспект;59.897584,30.512745",
    "b": "поселок Новосергиевка;59.906159,30.556786",
    "c": "Лондон молл;59.911953,30.446354",
    "d": "метро Дыбенко;59.907063,30.483070"
    }

### Tiles

In [ ]:
def here_tiles(**kwdict):
    """
    Return a HERE map tiles URL, based on default values that can be 
    overwritten by kwdict.
    """
    params = dict(
        app_id     = config.appid,
        app_code   = config.appcode,
        scheme     = 'reduced.day',
        tilesize   = '256',
        tileformat = 'png8',
        lg         = 'rus',
        x          = '{x}',
        y          = '{y}',
        z          = '{z}',
        )

    params.update(kwdict)

    url = (
        'https://2.base.maps.api.here.com/maptile/2.1/maptile/newest/{scheme}/'
        '{z}/{x}/{y}/{tilesize}/{tileformat}'
        '?lg={lg}&app_id={app_id}&app_code={app_code}'
        ).format(**params)
    return url

### Waypoint Sequence

In [ ]:
def get_waypoints(points):
    """
    Return points in optimal order.
    """
    params = {
        "app_id": config.appid,
        "app_code": config.appcode,
        "start": points["a"],
        "destination1": points["b"],
        "destination2": points["c"],
        "end": points["d"],
        "mode": "fastest;car"
        }
    
    url = 'https://wse.api.here.com/2/findsequence.json'
    
    response = requests.get(url, params=params)
    waypoints = response.json()
    
    return waypoints  

### Route

In [ ]:
def get_route(waypoints):
    """
    Return a road with all geometry.
    """
    waypoint0 = '{0},{1}'.format(
                            waypoints["results"][0]["waypoints"][0]["lat"],
                            waypoints["results"][0]["waypoints"][0]["lng"]
                            )
    waypoint1 = '{0},{1}'.format(
                            waypoints["results"][0]["waypoints"][1]["lat"],
                            waypoints["results"][0]["waypoints"][1]["lng"]
                            )
    waypoint2 = '{0},{1}'.format(
                            waypoints["results"][0]["waypoints"][2]["lat"],
                            waypoints["results"][0]["waypoints"][2]["lng"]
                            )
    waypoint3 = '{0},{1}'.format(
                            waypoints["results"][0]["waypoints"][3]["lat"],
                            waypoints["results"][0]["waypoints"][3]["lng"]
                            )
    
    params = {
        "app_id": config.appid,
        "app_code": config.appcode,
        "waypoint0": waypoint0,
        "waypoint1": waypoint1,
        "waypoint2": waypoint2,
        "waypoint3": waypoint3,
        "mode": "fastest;car;traffic:enabled",
        "representation": "display"
        }

    url = 'https://route.api.here.com/routing/7.2/calculateroute.json'
    
    response = requests.get(url, params=params)
    route = response.json()
    
    return route

## Map initializing

In [ ]:
# Build a HERE tiles
tiles = here_tiles()

# Initialize the map object
map = folium.Map(
        location=[59.925658, 30.331779],  
        zoom_start=11, 
        tiles=tiles, 
        attr='here.com'
        )

## Route to line

In [ ]:
# Build of a route based on waypoints data
route = get_route(get_waypoints(points))

# Build of a line for represent a route on the map
line = []

for leg in route["response"]["route"][0]["leg"]:
    for shape in leg["maneuver"]:
        for point in shape["shape"]:
            mpoint = point.split(',')
            line.append([float(mpoint[0]), float(mpoint[1])])

## Route on the map

In [ ]:
# Just an icon
iurl = 'https://img.icons8.com/plasticine/100/000000/gps-device.png'

# Add points to map
for point in points.values():
    point = str(point.split(';')[1]).split(',')
    point = [float(point[0]), float(point[1])]
    folium.Marker(point, icon=folium.features.CustomIcon(iurl, icon_size=(35,35))).add_to(map)

# Route to the map
folium.PolyLine(line, color="green", weight=6, opacity=1).add_to(map)

# Show the map
map